In [1]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
import os
from dotenv import load_dotenv
from selenium.webdriver.common.keys import Keys
load_dotenv()

True

In [2]:
my_email = os.getenv("email")
my_password = os.getenv("password")

In [75]:
def extract_reddit_posts(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)

    # Wait for the feed to load
    WebDriverWait(driver, 10)
    
    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(6)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break


    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    posts = soup.find_all("div", {"class": "_1poyrkZ7g36PawDueRza-J"})

    rows = []
    for post in posts:
        link = post.find_all('a')
        try:
            linkurl = 'https://www.reddit.com' + re.search(r"/r/(\w+)/comments/(\w+)/(\w+)/", str(link)).group()
        except:
            linkurl = 'Error loading'
        post_title = post.find('h3', class_='_eYtD2XCVieq6emjKBH3m')
        rows.append([linkurl, post_title.text])
            
        rows.append([linkurl, post_title.text])

    driver.quit()

    return pd.DataFrame(rows, columns=['Link', "Title"])


In [76]:
df_reddit1 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link')
df_reddit1.drop_duplicates(inplace=True)
df_reddit1.to_csv('df_reddit1.csv', index = False)


In [21]:
df_reddit2 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=comments')
df_reddit2.drop_duplicates(inplace=True)
df_reddit2.to_csv('df_reddit2.csv', index = False)


In [22]:
df_reddit3 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=new')
df_reddit3.drop_duplicates(inplace=True)
df_reddit3.to_csv('df_reddit3.csv', index = False)


In [23]:
df_reddit4 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=top')
df_reddit4.drop_duplicates(inplace=True)
df_reddit4.to_csv('df_reddit4.csv', index = False)


In [24]:
df_reddit5 = extract_reddit_posts('https://www.reddit.com/search/?q=Artificial%20Intelligence&type=link&sort=hot')
df_reddit5.drop_duplicates(inplace=True)
df_reddit5.to_csv('df_reddit5.csv', index = False)


In [111]:
df_reddit1.to_csv('df_reddit1.csv', index = False)

In [112]:
df_red1

,index,Link,Title
0,0,https://www.reddit.com/r/Futurology/comments/1...,ChatGPT is just the beginning: Artificial inte...
1,2,https://www.reddit.com/r/conspiracy/comments/1...,We need to discuss Artificial Intelligence. As...
2,4,https://www.reddit.com/r/IAmA/comments/10pi1d4...,"I'm Professor Toby Walsh, a leading artificial..."
3,6,https://www.reddit.com/r/Futurology/comments/1...,US experts warn AI likely to kill off jobs – a...
4,8,https://www.reddit.com/r/Futurology/comments/1...,Jobs With the Lowest Risk of Automation by Art...


In [51]:
df_reddit

,Content,Author,Date,upvotes,comments,awards


In [86]:
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

def extract_reddit_post_info(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    post_container = soup.find("div", {"data-testid": "post-container"})

    title = post_container.find("h1", {"class": "_eYtD2XCVieq6emjKBH3m"})
    username_link = post_container.find("a", {"class": "_2tbHP6ZydRpjI44J3syuqC"})
    username = username_link.text.strip("u/")
    message = post_container.find("div", {"class": "_292iotee39Lmt0MkQZ2hPV RichTextJSON-root"}).text
    date = post_container.find("span", {"class": "_2VF2J19pUIMSLJFky-7PEI"}).text
    upvotes = post_container.find("div", {"class": "_1E9mcoVn4MYnuBQSVDt1gC"}).text
    comments = post_container.find("span", {"class": "FHCV02u6Cp2zYL0fhQPsO"}).text

    driver.quit()

    return pd.DataFrame([[title, username, message, date, upvotes, comments]], columns=["Title", "Username", "Message", "Date", "Upvotes", "Comments"])


In [87]:
df = extract_reddit_post_info("https://www.reddit.com/r/conspiracy/comments/109ew7w/we_need_to_discuss_artificial_intelligence_as_a/")
print(df)


                                               Title      Username  \
0  [We need to discuss Artificial Intelligence. A...  Hodlthehodl1   

                                             Message          Date Upvotes  \
0  To be clear, I'm not talking about the skynet ...  2 months ago     916   

       Comments  
0  466 comments  


In [88]:
df

,Title,Username,Message,Date,Upvotes,Comments
0,[We need to discuss Artificial Intelligence. A...,Hodlthehodl1,"To be clear, I'm not talking about the skynet ...",2 months ago,916,466 comments


In [102]:
def extract_reddit_post_info(df):
    reddit_comments = []

    for i in range(len(df)):
        url = df['Link'][i]
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        #options.add_argument("--headless")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)

        html = driver.execute_script("return document.body.outerHTML;")
        soup = BeautifulSoup(html, "html.parser")

        post_container = soup.find("div", {"data-testid": "post-container"})

        try:
            title = post_container.find("h1", {"class": "_eYtD2XCVieq6emjKBH3m"})
            username_link = post_container.find("a", {"class": "_2tbHP6ZydRpjI44J3syuqC"})
            username = username_link.text.strip("u/")
            message = post_container.find("div", {"class": "_292iotee39Lmt0MkQZ2hPV RichTextJSON-root"}).text
            date = post_container.find("span", {"class": "_2VF2J19pUIMSLJFky-7PEI"}).text
            upvotes = post_container.find("div", {"class": "_1E9mcoVn4MYnuBQSVDt1gC"}).text
            comments = post_container.find("span", {"class": "FHCV02u6Cp2zYL0fhQPsO"}).text
        except:
            pass

        driver.quit()

        reddit_comments.append([title, username, message, date, upvotes, comments, url])

        print(f'{url} done')

    return pd.DataFrame(reddit_comments, columns=["Title", "Username", "Message", "Date", "Upvotes", "Comments", "URL"])


In [113]:
extract_reddit_post_info(df_red1)

https://www.reddit.com/r/Futurology/comments/10qvt8l/chatgpt_is_just_the_beginning_artificial/ done
https://www.reddit.com/r/conspiracy/comments/109ew7w/we_need_to_discuss_artificial_intelligence_as_a/ done
https://www.reddit.com/r/IAmA/comments/10pi1d4/im_professor_toby_walsh_a_leading_artificial/ done
https://www.reddit.com/r/Futurology/comments/10wx3tc/us_experts_warn_ai_likely_to_kill_off_jobs_and/ done
https://www.reddit.com/r/Futurology/comments/11ck3fb/jobs_with_the_lowest_risk_of_automation_by/ done


,Title,Username,Message,Date,Upvotes,Comments,URL
0,[ChatGPT is just the beginning: Artificial int...,Gari_305,,1 month ago,14.9k,2.1k comments,https://www.reddit.com/r/Futurology/comments/1...
1,[We need to discuss Artificial Intelligence. A...,Hodlthehodl1,"To be clear, I'm not talking about the skynet ...",2 months ago,908,466 comments,https://www.reddit.com/r/conspiracy/comments/1...
2,"[I'm Professor Toby Walsh, a leading artificia...",nsw,"Hi Reddit, Prof Toby Walsh here, keen to chat ...",1 month ago,4.9k,1.2k comments,https://www.reddit.com/r/IAmA/comments/10pi1d4...
3,[US experts warn AI likely to kill off jobs – ...,Gari_305,,1 month ago,2.5k,557 comments,https://www.reddit.com/r/Futurology/comments/1...
4,[Jobs With the Lowest Risk of Automation by Ar...,WhiteyKC,,15 days ago,423,351 comments,https://www.reddit.com/r/Futurology/comments/1...


In [114]:
def extract_reddit_post_comments(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    feed = soup.find('shreddit-comment-tree', {'id':"comment-tree"})
    post_container = feed.find("delayed-dialog", {"class": 'visible'})

    for post in post_container:
        text = post.find('div', {'id': '-post-rtjson-content'})

    driver.quit()

    return pd.DataFrame([[text]], columns=["text"])


In [143]:

def extract_reddit_post_comments(df):
    reddit_post_comments = []

    for i in range(len(df)):
        url = df['Link'][i]
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        #options.add_argument("--headless")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)

        time.sleep(5)

        while True:
            scroll_height = 10
            document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
            driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
            time.sleep(3)
            document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
            if document_height_after == document_height_before:
                break

        html = driver.execute_script("return document.body.outerHTML;")
        soup = BeautifulSoup(html, 'html.parser')
        driver.quit()
        posts = soup.find_all('div', {'class': '_3tw__eCCe7j-epNCKGXUKk'})

        for post in posts:
            try:
                name = post.find('a', {'class': 'wM6scouPXXsFDSZmZPHRo DjcdNGtVXPcxG0yiFXIoZ _23wugcdiaj44hdfugIAlnX'}).text
            except:
                name = post.find('a', {'class': 'wM6scouPXXsFDSZmZPHRo DjcdNGtVXPcxG0yiFXIoZ _23wugcdiaj44hdfugIAlnX'})

            try:
                date = post.find('a', class_='_3yx4Dn0W3Yunucf5sVJeFU').text
            except:
                date = post.find('a', class_='_3yx4Dn0W3Yunucf5sVJeFU')
            
            try:
                upvotes = post.find('div', {'class':"_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ _3ChHiOyYyUkpZ_Nm3ZyM2M"}).text
            except:
                upvotes = post.find('div', {'class':"_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ _3ChHiOyYyUkpZ_Nm3ZyM2M"})

            try:
                comment_thread = post.find('p', class_='_1qeIAgB0cPwnLhDF9XSiJM').text
            except:
                comment_thread = post.find('p', class_='_1qeIAgB0cPwnLhDF9XSiJM')
            reddit_post_comments.append([name, date, upvotes, comment_thread, url])

    df_reddit_post_comments2 = pd.DataFrame(reddit_post_comments, columns=["Name", 'date', 'upvotes', 'comment2', 'url'])
    return df_reddit_post_comments2




In [144]:
extract_reddit_post_comments(df_red1)

,Name,date,upvotes,comment2,url
0,FuturologyBot,1 mo. ago,Vote,The following submission statement was provide...,https://www.reddit.com/r/Futurology/comments/1...
1,CaptPants,1 mo. ago,4.7k,I hope it's used for more than just cutting jo...,https://www.reddit.com/r/Futurology/comments/1...
2,Shanhaevel,1 mo. ago,2.0k,"Haha, that's rich. As if.",https://www.reddit.com/r/Futurology/comments/1...
3,SatiricalComment,1 mo. ago,1.1k,One of the intents of many scientists who deve...,https://www.reddit.com/r/Futurology/comments/1...
4,fistfulloframen,1 mo. ago,30,You can use it to fix up your resume after you...,https://www.reddit.com/r/Futurology/comments/1...
...,...,...,...,...,...
705,HairyChest69,14 days ago,1,I always wonder about retail workers. The peop...,https://www.reddit.com/r/Futurology/comments/1...
706,AnEggscellentName,14 days ago,None,None,https://www.reddit.com/r/Futurology/comments/1...
707,TheWaterDoctor,14 days ago,1,Building services and maintenance.,https://www.reddit.com/r/Futurology/comments/1...
708,Unrigg3D,14 days ago,1,Nobody is at risk for not having jobs. This is...,https://www.reddit.com/r/Futurology/comments/1...
